# <span style="color:#84b6f4;">Detección de comentarios de hate</span>

### <span style="color:#77dd77;">Formación del DataFrame</span>

En primer lugar, sacamos los datos del fichero txt y formamos un DataFrame con ellos:

In [1]:
import pandas as pd
import os
from nltk.corpus import wordnet
import nltk
import random

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def synonym_replacement(sentence, n):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n: # n define el número de palabras a reemplazar
            break

    sentence = ' '.join(new_words)
    return sentence

# Especificamos el archivo de texto
txt_file = os.getcwd()+'/resources/labeled_corpus_6K.txt'

# Leemos el archivo de texto con el formato específico
data = []
with open(txt_file, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split(';||;')
        if len(parts) == 3:
            data.append(parts)

# Creamos un DataFrame a partir de los datos
column_names = ['id', 'text', 'hate']
data_frame = pd.DataFrame(data, columns=column_names)

# Convertimos la columna 'hate' en enteros
data_frame['hate'] = data_frame['hate'].astype(int)

# Seleccionamos las filas con 'hate=0'
hate_1_rows = data_frame[data_frame['hate'] == 1].copy()

# Generamos filas aumentadas
augmented_rows = []
for index, row in hate_1_rows.iterrows():
    new_text_1 = synonym_replacement(row['text'], 2)
    augmented_row_1 = [row['id'], new_text_1, row['hate']]
    new_text_2 = synonym_replacement(row['text'], 2)
    augmented_row_2 = [row['id'], new_text_2, row['hate']]

    if new_text_1 != new_text_2:
        augmented_rows.append(augmented_row_1)
        augmented_rows.append(augmented_row_2)
    else:
        augmented_rows.append(augmented_row_1)

# Creamos un DataFrame con las filas aumentadas
augmented_data_frame = pd.DataFrame(augmented_rows, columns=column_names)

# Concatenamos el DataFrame original con las filas aumentadas
balanced_data_frame = pd.concat([data_frame, augmented_data_frame])

# Mostramos las primeras filas del DataFrame aumentado y balanceado
print(balanced_data_frame.head(5))

print(balanced_data_frame[balanced_data_frame['hate'] == 0].shape[0])
print(balanced_data_frame[balanced_data_frame['hate'] == 1].shape[0])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fjmon\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\fjmon\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


                      id                                               text  \
0  id=828025263321657348  Ismael es egocentrico porque se vuelve loca si...   
1  id=828025128797741057  ..ya tardaba en salir quien pronunciase nombre...   
2  id=828025087815274496  (Esto no es un discurso político y razonado, o...   
3  id=828025006626058241  Muy despreciados,siiii,pero todos vestidos de ...   
4  id=828024709761658880  marica explicame porque a veces no te entiendo...   

   hate  
0     0  
1     0  
2     0  
3     1  
4     1  
4406
4674


### <span style="color:#77dd77;">Entrenamiento</span>

Seleccionamos las métricas que usaremos para predecir, así como el atributo objetivo que, en este caso, será numérico.

In [2]:
# Atributo que usaremos para predecir
attributes = balanced_data_frame[['text']]

# Atributo objetivo a predecir
goal = balanced_data_frame['hate']

A continuación, entrenamos un modelo de clasificación con una red neuronal:

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

text = balanced_data_frame['text']

# Dividimos los datos en conjuntos de entrenamiento (70%) y prueba (30%)
attributes_train, attributes_test, goal_train, goal_test = train_test_split(text, goal, test_size=0.3, random_state=42)

# Creamos un vectorizador de texto
vectorizer = CountVectorizer()
tweet_train_vectorized = vectorizer.fit_transform(attributes_train)
tweet_test_vectorized = vectorizer.transform(attributes_test)

# Entrenamos el modelo de clasificación de redes neuronales
mlp_classifier = MLPClassifier(hidden_layer_sizes=(50, 50, 50), activation='relu', solver='adam', random_state=42, max_iter=1000)
mlp_classifier.fit(tweet_train_vectorized, goal_train)

# Realizamos predicciones con el conjunto de prueba
prediction = mlp_classifier.predict(tweet_test_vectorized)

# Calculamos la precisión y F1-Score del modelo
accuracy = accuracy_score(goal_test, prediction)
f1 = f1_score(goal_test, prediction)
print("Precisión del modelo en el conjunto de prueba:", accuracy)
print("F1-Score:", f1)

# Mostramos la matriz de confusión
print(confusion_matrix(goal_test, prediction))

Precisión del modelo en el conjunto de prueba: 0.9030837004405287
F1-Score: 0.9085239085239085
[[1149  196]
 [  68 1311]]


Probamos el modelo con algunos ejemplos concretos:

In [4]:
message = ["respetuoso", "de hate"]

# Texto de ejemplo 1
example_text_1 = "Vaya que eres cortito, ¿eh?"
example_text_1_vectorized = vectorizer.transform([example_text_1])

# Texto de ejemplo 2
example_text_2 = "Es la peor película que he visto en los últimos años."
example_text_2_vectorized = vectorizer.transform([example_text_2])

# Texto de ejemplo 3
example_text_3 = "Si te gusta esta peli es que eres de lo más tonto."
example_text_3_vectorized = vectorizer.transform([example_text_3])

# Texto de ejemplo 4
example_text_4 = "Te aprecio mucho."
example_text_4_vectorized = vectorizer.transform([example_text_4])

# Texto de ejemplo 5
example_text_5 = "Te quiero, pero eres una basura."
example_text_5_vectorized = vectorizer.transform([example_text_5])

print("El comentario '" + example_text_1 + "' es " + message[mlp_classifier.predict(example_text_1_vectorized)[0]] + ".")
print("El comentario '" + example_text_2 + "' es " + message[mlp_classifier.predict(example_text_2_vectorized)[0]] + ".")
print("El comentario '" + example_text_3 + "' es " + message[mlp_classifier.predict(example_text_3_vectorized)[0]] + ".")
print("El comentario '" + example_text_4 + "' es " + message[mlp_classifier.predict(example_text_4_vectorized)[0]] + ".")
print("El comentario '" + example_text_5 + "' es " + message[mlp_classifier.predict(example_text_5_vectorized)[0]] + ".")

El comentario 'Vaya que eres cortito, ¿eh?' es respetuoso.
El comentario 'Es la peor película que he visto en los últimos años.' es respetuoso.
El comentario 'Si te gusta esta peli es que eres de lo más tonto.' es de hate.
El comentario 'Te aprecio mucho.' es respetuoso.
El comentario 'Te quiero, pero eres una basura.' es de hate.


Para finalizar, exportamos el modelo:

In [5]:
from joblib import dump

# Guardamos el modelo
model_filename = 'hate_classifier.joblib'
dump(mlp_classifier, model_filename)

# Guardamos el vectorizador
vectorizer_filename = 'hate_vectorizer.joblib'
dump(vectorizer, vectorizer_filename)

['hate_vectorizer.joblib']